In [40]:
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import pylab as pl
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import tabulate
from operator import add
import itertools

from shapely import wkt
from shapely.geometry import shape, LineString, Point
    
# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

In [41]:
flujos_loc = pd.read_csv('tablas/dd_localidades.csv')
print(flujos_loc.shape[0])
flujos_loc.head(3)

377610


,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.0,583715
1,10202521,1020,2521,1304729,12200,90.0,612802
2,10202522,1020,2522,1304729,2659,17.0,617076


In [42]:
locs = cargar_datos_geo()[2][['CODLOC', 'geometry']]
locs.head(3)

/home/guillermo/anaconda3/envs/tesis/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


,CODLOC,geometry
0,2220,POINT (549273.219 6633915.063)
1,2521,POINT (442454.671 6652273.541)
2,2522,POINT (457921.001 6633361.660)


In [43]:
locs_cod = list(locs.CODLOC)
len(locs_cod) ** 2 - len(locs_cod)

377610

In [45]:
# pega geometrías
dd = flujos_loc.merge(locs, left_on='cod_ori', right_on='CODLOC')
dd = dd.merge(locs, left_on='cod_des', right_on='CODLOC')
dd.drop(['CODLOC_x', 'CODLOC_y'], axis=1, inplace=True)
dd.rename({'geometry_x':'geom_ori', 'geometry_y':'geom_des'}, axis=1, inplace=True)

dd.head()

# exporta CSV con geometrías puntuales para mapa de flujos
dd.to_csv('capas/loc_lines.csv', index=False)

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m,geom_ori,geom_des
0,10202220,1020,2220,1304729,40657,635.0,583715,POINT (573082.248 6145008.434),POINT (549273.219 6633915.063)
1,25212220,2521,2220,12200,40657,NaN,137927,POINT (442454.671 6652273.541),POINT (549273.219 6633915.063)
2,25222220,2522,2220,2659,40657,NaN,111408,POINT (457921.001 6633361.660),POINT (549273.219 6633915.063)
3,26212220,2621,2220,2531,40657,NaN,106867,POINT (468458.270 6601361.867),POINT (549273.219 6633915.063)
4,27212220,2721,2220,380,40657,NaN,58392,POINT (502880.928 6648162.053),POINT (549273.219 6633915.063)


In [18]:
print(flujos_loc.shape)
flujos_loc.head()

(8791, 7)


,loc_origen,loc_destino,personas_mig,pobl_origen,pobl_destino,cod,distancia_m
0,1020,2220,635,1304729,40657,10202220,583715
1,3221,2220,5,71258,40657,32212220,570608
2,3320,2220,10,19865,40657,33202220,547145
3,3321,2220,1,20524,40657,33212220,573837
4,3322,2220,1,25947,40657,33222220,583158


In [16]:
matrix = pd.pivot_table(flujos_loc,
                        index ='loc_origen',
                        columns='loc_destino',
                        fill_value=0,
                        margins=True,
                        margins_name='Total')
matrix

cod                                            \
loc_destino          1020          2220          2521          2522   
loc_origen                                                            
1020         0.000000e+00  1.020222e+07  1.020252e+07  1.020252e+07   
2220         2.220102e+07  0.000000e+00  0.000000e+00  0.000000e+00   
2521         2.521102e+07  0.000000e+00  0.000000e+00  0.000000e+00   
2522         2.522102e+07  0.000000e+00  0.000000e+00  0.000000e+00   
2621         2.621102e+07  0.000000e+00  0.000000e+00  0.000000e+00   
...                   ...           ...           ...           ...   
19941        1.994110e+08  0.000000e+00  0.000000e+00  0.000000e+00   
19942        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
19947        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
19960        1.996010e+08  0.000000e+00  0.000000e+00  0.000000e+00   
Total        1.016396e+08  1.047701e+08  9.971067e+07  1.054304e+08   

                                                                        \
loc_destino          2621          2724       2725      2822      2823   
loc_origen                                                               
1020         1.020262e+07  1.020272e+07   10202725  10202822  10202823   
2220         0.000000e+00  0.000000e+00          0         0         0   
2521         0.000000e+00  0.000000e+00          0         0         0   
2522         0.000000e+00  0.000000e+00          0         0         0   
2621         0.000000e+00  0.000000e+00          0         0         0   
...                   ...           ...        ...       ...       ...   
19941        0.000000e+00  0.000000e+00          0         0         0   
19942        0.000000e+00  0.000000e+00          0         0         0   
19947        0.000000e+00  0.000000e+00          0         0         0   
19960        0.000000e+00  0.000000e+00          0         0         0   
Total        1.355843e+08  1.105461e+08  134632725  74205322  23762823   

                           ... pobl_origen                                \
loc_destino          2828  ...       19962  19963    19964  19965  19966   
loc_origen                 ...                                             
1020         1.020283e+07  ...           0      0     0.00      0      0   
2220         0.000000e+00  ...           0      0     0.00      0      0   
2521         0.000000e+00  ...           0      0     0.00      0      0   
2522         0.000000e+00  ...           0      0     0.00      0      0   
2621         0.000000e+00  ...           0      0     0.00      0      0   
...                   ...  ...         ...    ...      ...    ...    ...   
19941        0.000000e+00  ...           0      0     0.00      0      0   
19942        0.000000e+00  ...           0      0     0.00      0      0   
19947        0.000000e+00  ...           0      0     0.00      0      0   
19960        0.000000e+00  ...           0      0     0.00      0      0   
Total        1.066213e+08  ...       62590  33217  6068.25  51830  15223   

                                                                     
loc_destino         19967     19968  19969      19971         Total  
loc_origen                                                           
1020             0.000000      0.00      0  1304729.0  1.304729e+06  
2220             0.000000      0.00      0        0.0  4.065700e+04  
2521             0.000000      0.00      0        0.0  1.220000e+04  
2522             0.000000      0.00      0        0.0  2.659000e+03  
2621             0.000000      0.00      0        0.0  2.531000e+03  
...                   ...       ...    ...        ...           ...  
19941            0.000000      0.00      0        0.0  6.800000e+01  
19942            0.000000      0.00      0        0.0  8.200000e+01  
19947            0.000000      0.00      0        0.0  1.600000e+01  
19960            0.000000      0.00      0        0.0  5.400000e+01  
Total        22